<a href="https://colab.research.google.com/github/e8558/Text2SQL/blob/main/Code/Recure_Chatbot_V3_all_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import PreTrainedTokenizerFast
from transformers import GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup


import os
import argparse
import pandas as pd
from tqdm import tqdm # 진행률 표시

In [3]:
# prompt: google drive mount

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# 작업 디렉토리 변경
os.chdir('/content/drive/MyDrive/Colab Notebooks/Project 2/Recruit Chatbot')
!pwd

/content/drive/MyDrive/Colab Notebooks/Project 2/Recruit Chatbot


In [5]:
from sklearn.model_selection import train_test_split

train_df = pd.read_csv('Dataset/Training/Interview_train.csv')
test_df = pd.read_csv('Dataset/Validation/Interview_val.csv')

cols = ['rawDataInfo.question.audioPath', 'dataSet.info.gender', 'dataSet.info.experience', 'dataSet.question.raw.text', 'dataSet.answer.raw.text']
train_df = train_df[cols]
test_df = test_df[cols]

job_dict = {'SM':'영업마케팅', 'PS':'공공서비스','RND':'연구개발', 'ICT':'정보통신','ARD':'디자인', 'MM':'생산관리', 'BM': '경영사무' }
train_df['domain'] = train_df['rawDataInfo.question.audioPath'].astype(str).apply(lambda x: x.split('/')[5].split('_')[2]).astype(str)
test_df['domain'] = test_df['rawDataInfo.question.audioPath'].astype(str).apply(lambda x: x.split('/')[5].split('_')[2]).astype(str)

train_df['domain'] = train_df['domain'].apply(lambda x: job_dict[x.upper()])
test_df['domain'] = test_df['domain'].apply(lambda x: job_dict[x.upper()])

train_df['dataSet.info.experience'] = train_df['dataSet.info.experience'].apply(lambda x: '신입' if x == 'NEW' else '경력')
test_df['dataSet.info.experience'] = test_df['dataSet.info.experience'].apply(lambda x: '신입' if x == 'NEW' else '경력')

train_df = train_df.drop(labels='rawDataInfo.question.audioPath', axis=1)
test_df = test_df.drop(labels='rawDataInfo.question.audioPath', axis=1)

train_df, val_df = train_test_split(train_df, test_size=0.12, random_state=42)
train_df.head()

<ipython-input-5-211f83aef90c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['domain'] = train_df['rawDataInfo.question.audioPath'].astype(str).apply(lambda x: x.split('/')[5].split('_')[2]).astype(str)
<ipython-input-5-211f83aef90c>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['domain'] = train_df['domain'].apply(lambda x: job_dict[x.upper()])
<ipython-input-5-211f83aef90c>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

,dataSet.info.gender,dataSet.info.experience,dataSet.question.raw.text,dataSet.answer.raw.text,domain
58352,FEMALE,신입,현대인은 하루 평균 세 번 거짓말을 하고 산다고 합니다 지원자님께서 해보셨던 가장 ...,네. 현대인은 하루 평균 세 번의 거짓말을 하고 산다고 하는데 음 저는 거짓말을 하...,경영사무
63544,FEMALE,신입,본인이 생각하는 십 년 후 미래는 어떨 것 같나요 만약 십 년 후에도 저희 회사를 ...,제 십 년 후 미래를 떠올린다면 저는 계속 이 회사에 다닐 것 같습니다. 저는 제가...,경영사무
43643,MALE,경력,면접자님께서는 동물 실험이 인간에게 주는 이익에 관해 말씀해 주시겠습니까 또 동물 ...,동물 실험을 반대하는 사람들은 인간이 동물을 확대한다고 생각합니다. 하지만 인간이 ...,영업마케팅
6930,FEMALE,신입,면접자님은 회사의 덕목이 무엇이라고 생각하시나요 어떤 회사가 좋은 회사라고 생각하시...,회사의 덕목은 근로자를 가족과 같이 여기는 회사라고 생각합니다. 근로자가 회사에 입...,공공서비스
56537,FEMALE,경력,스트레스를 받으면 어떻게 하시나요 왜 그런 방법으로 스트레스를 해소하시는지도 함께 ...,저는 평소에 스트레스들을 받으면 그런 스트레스들을 집으로 가져가는 게 아니라 밖에서...,영업마케팅


In [6]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
bos_token='</s>', eos_token='</s>', unk_token='<unk>',
pad_token='<pad>', mask_token='<mask>')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [7]:
class QADataset(Dataset):
    def __init__(self, dataset, tokenizer, max_length=256):
        self.dataset = dataset # This is the dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        gender = self.dataset.iloc[idx]['dataSet.info.gender']
        experience = self.dataset.iloc[idx]['dataSet.info.experience']
        field = self.dataset.iloc[idx]['domain']
        question = self.dataset.iloc[idx]['dataSet.question.raw.text']
        answer = self.dataset.iloc[idx]['dataSet.answer.raw.text']

        input_text = f"<s> 질문: {question} 경력: {experience} 직업: {field} 답변: {answer} </s>"

        encoding = self.tokenizer(
            input_text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        # labels를 input_ids의 복사본으로 생성
        labels = input_ids.clone()

        # 패딩 토큰에 대한 손실을 무시하기 위해 패딩 토큰 인덱스를 -100으로 설정
        labels[input_ids == self.tokenizer.pad_token_id] = -100

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }

In [8]:
# Dataset과 DataLoader 생성
train_dataset = QADataset(train_df, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

val_dataset = QADataset(val_df, tokenizer)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=True)

test_dataset = QADataset(test_df, tokenizer)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# DataLoader에서 배치 샘플 확인
for batch in train_dataloader:
    print(batch['input_ids'].shape)  # (batch_size, max_length)
    print(batch['attention_mask'].shape)  # (batch_size, max_length)
    print(batch['labels'].shape)  # (batch_size, max_length)
    break

torch.Size([16, 256])
torch.Size([16, 256])
torch.Size([16, 256])


# Train

In [ ]:
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [ ]:
# 체크포인트를 저장할 경로 설정
checkpoint_dir = "./checkpoints_all_layer_train"
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

In [ ]:
from tqdm.notebook import tqdm

num_epochs = 5

# 옵티마이저와 스케줄러 정의
optimizer = AdamW(model.parameters(), lr=5e-5)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# 학습 루프
best_valid_loss = float('inf')  # 최적의 검증 손실 저장용
for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0
    train_steps = 0

    train_pbar = tqdm(train_dataloader, desc=f'Training Epoch {epoch+1}')
    for batch in train_pbar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        labels=labels)
        loss = outputs.loss

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # 그래디언트 클리핑

        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        total_train_loss += loss.item()
        train_steps += 1

        # 현재 loss를 tqdm에 표시
        train_pbar.set_postfix({'loss': f'{loss.item():.4f}'})

    avg_train_loss = total_train_loss / train_steps
    print(f"\nEpoch {epoch+1} Average Training Loss: {avg_train_loss:.4f}")
    # 에포크가 끝날 때마다 체크포인트 저장
    epoch_checkpoint_path = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch + 1}")
    model.save_pretrained(epoch_checkpoint_path)
    tokenizer.save_pretrained(epoch_checkpoint_path)
    print(f"Checkpoint saved at {epoch_checkpoint_path}")

    model.eval()
    total_valid_loss = 0
    valid_steps = 0
    with torch.no_grad():  # 그래디언트 계산 비활성화
        valid_pbar = tqdm(val_dataloader, desc=f'Validation Epoch {epoch+1}')
        for batch in valid_pbar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids,
                          attention_mask=attention_mask,
                          labels=labels)
            loss = outputs.loss

            total_valid_loss += loss.item()
            valid_steps += 1

            valid_pbar.set_postfix({'loss': f'{loss.item():.4f}'})

    avg_valid_loss = total_valid_loss / valid_steps
    print(f"Epoch {epoch+1} Average Validation Loss: {avg_valid_loss:.4f}")
    # 최적의 모델 저장
    if avg_valid_loss < best_valid_loss:
        best_valid_loss = avg_valid_loss
        print(f"Saving best model with validation loss: {best_valid_loss:.4f}")
        epoch_checkpoint_path = os.path.join(checkpoint_dir, f"best_model")
        model.save_pretrained(epoch_checkpoint_path)
        tokenizer.save_pretrained(epoch_checkpoint_path)

print("\nTraining completed!")

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Training Epoch 1:   0%|          | 0/3745 [00:00<?, ?it/s]


Epoch 1 Average Training Loss: 3.0140
Checkpoint saved at ./checkpoints_all_layer_train/checkpoint_epoch_1


Validation Epoch 1:   0%|          | 0/511 [00:00<?, ?it/s]

Epoch 1 Average Validation Loss: 2.8395
Saving best model with validation loss: 2.8395


...

Epoch 4 Average Validation Loss: 2.7068
Saving best model with validation loss: 2.7068


Training Epoch 5:   0%|          | 0/3745 [00:00<?, ?it/s]


Epoch 5 Average Training Loss: 2.4640
Checkpoint saved at ./checkpoints_all_layer_train/checkpoint_epoch_5


Validation Epoch 5:   0%|          | 0/511 [00:00<?, ?it/s]

Epoch 5 Average Validation Loss: 2.7011
Saving best model with validation loss: 2.7011

Training completed!


# Test

In [9]:
!pip install bert_score
!pip install rouge

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.6 MB/s eta 0:00:00


In [10]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from bert_score import score as bert_score
from rouge import Rouge
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from tqdm.notebook import tqdm
import torch

# 저장된 모델과 토크나이저 로드
checkpoint_dir = "./checkpoints_all_layer_train/best_model"
model = GPT2LMHeadModel.from_pretrained(checkpoint_dir)
tokenizer = PreTrainedTokenizerFast.from_pretrained(checkpoint_dir)

# GPU 또는 CPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# BERTScore, ROUGE, BLEU 평가 함수
def evaluate_model(model, tokenizer, test_dataloader, device):
    model.eval()
    references = []
    candidates = []

    rouge = Rouge()
    total_bleu = []
    total_rouge = {"rouge-1": [], "rouge-2": [], "rouge-l": []}

    smoothing_function = SmoothingFunction().method1

    with torch.no_grad():
        test_pbar = tqdm(test_dataloader, desc='Testing')
        for batch in test_pbar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # 모델 예측
            generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_new_tokens=250)
            generated_texts = [tokenizer.decode(ids, skip_special_tokens=True) for ids in generated_ids]
            candidates.extend(generated_texts)

            original_texts = []
            for ids in labels:
                ids = ids[ids != -100]  # -100 값 제거
                decoded_text = tokenizer.decode(ids, skip_special_tokens=True)
                original_texts.append(decoded_text)
            references.extend(original_texts)

    # BERTScore 계산
    bert_p, bert_r, bert_f1 = bert_score(candidates, references, lang="ko")

    # ROUGE, BLEU 계산
    for ref, cand in zip(references, candidates):
        rouge_score = rouge.get_scores(cand, ref)[0]
        total_rouge["rouge-1"].append(rouge_score["rouge-1"]["f"])
        total_rouge["rouge-2"].append(rouge_score["rouge-2"]["f"])
        total_rouge["rouge-l"].append(rouge_score["rouge-l"]["f"])

        # BLEU 점수
        bleu = sentence_bleu([ref.split()], cand.split(), smoothing_function=smoothing_function)
        total_bleu.append(bleu)

    # 평균 계산
    avg_bert_f1 = torch.mean(bert_f1).item()
    avg_rouge_1 = torch.mean(torch.tensor(total_rouge["rouge-1"])).item()
    avg_rouge_2 = torch.mean(torch.tensor(total_rouge["rouge-2"])).item()
    avg_rouge_l = torch.mean(torch.tensor(total_rouge["rouge-l"])).item()
    avg_bleu = sum(total_bleu) / len(total_bleu)

    print("\nEvaluation Metrics:")
    print(f"BERTScore F1: {avg_bert_f1:.4f}")
    print(f"ROUGE-1: {avg_rouge_1:.4f}")
    print(f"ROUGE-2: {avg_rouge_2:.4f}")
    print(f"ROUGE-L: {avg_rouge_l:.4f}")
    print(f"BLEU: {avg_bleu:.4f}")

    return {
        "bert_score_f1": avg_bert_f1,
        "rouge-1": avg_rouge_1,
        "rouge-2": avg_rouge_2,
        "rouge-l": avg_rouge_l,
        "bleu": avg_bleu,
    }

# 모델 평가 실행
metrics = evaluate_model(model, tokenizer, test_dataloader, device)

Testing:   0%|          | 0/502 [00:00<?, ?it/s]

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='le

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]


Evaluation Metrics:
BERTScore F1: 0.9560
ROUGE-1: 0.9768
ROUGE-2: 0.9661
ROUGE-L: 0.9768
BLEU: 0.9293


In [30]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# 저장된 모델 경로
checkpoint_dir =  "./checkpoints_all_layer_train/best_model"

# 모델과 토크나이저 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GPT2LMHeadModel.from_pretrained(checkpoint_dir)
tokenizer = PreTrainedTokenizerFast.from_pretrained(checkpoint_dir)
model.to(device)

# 질문에 대한 답변 생성 함수
def generate_answer(question, model, tokenizer, device, max_new_tokens=50):
    """
    학습된 모델로 질문에 대한 답변을 생성합니다.

    Args:
        question (str): 사용자 입력 질문
        model: 학습된 GPT2 모델
        tokenizer: 학습된 모델의 토크나이저
        device: 모델 실행 장치 (CPU 또는 GPU)
        max_new_tokens (int): 생성할 최대 토큰 수

    Returns:
        str: 모델이 생성한 답변
    """
    # 입력 질문 토큰화
    inputs = tokenizer.encode(question, return_tensors="pt").to(device)

    # 모델로 답변 생성
    outputs = model.generate(
        inputs,
        max_new_tokens=200,
        eos_token_id=None,
        no_repeat_ngram_size=2,  # 반복 방지
        num_return_sequences=1,  # 한 개의 답변 생성
        temperature=0.7,         # 창의성 제어
        top_k=50,                # 상위 k개만 샘플링
        top_p=0.9,               # 누적 확률 기반 샘플링
    )

    print(outputs)

    # 생성된 답변 디코딩
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer


# 예제 질문
question = "갈등 상황을 해결한 경험을 이야기해주세요"
experience = '신입'
field = '공공서비스'
input_text = f"<s> 질문: {question} 경력: {experience} 직업: {field}"

# 답변 생성
answer = generate_answer(input_text, model, tokenizer, device)

# 답변에서 '답변: ' 이후의 텍스트 추출
if '답변: ' in answer:
    answer_text = answer.split('답변: ')[1]
else:
    answer_text = answer

# '.' 기준으로 문장 분리 후 마지막 문장 제외
final_result = '. '.join(answer_text.split('.')[:-1]).strip()

# 출력 결과
print(f"질문: {question}")
print(f"답변: {answer_text}")
print(f". 기준으로 마무리한 결과: {final_result}.")

tensor([[    0, 24454,   401, 11711, 13771, 10670,  8704, 15902, 11120, 15084,
          7801,  8084, 12170,   401, 33442, 10888,   401, 11599, 16371,   739,
             1,     0, 12817,  7643,   401,  9265,  7162, 10655, 12961, 13717,
         21835, 12498, 10293,  8704, 22719, 32987,  9423,  9265,  8806,  9228,
          7162, 14253,  6903, 10364,  7285, 18264, 42863,  9442,  7788, 10293,
          8718, 16691,  9265, 22633, 14253, 14160, 10364,  9675,  9304, 42863,
         23029,  7788, 21835, 17053, 10107,  8718, 16693, 25856,  9265, 26620,
         14253, 31582,  9676,  9080, 12396,  9177, 21087, 14160,  8022,  9167,
         28292, 10667,  8718,  8017, 16691, 11403,  9265, 21062, 14253, 13717,
         34686,  9919,  9306, 14160,  8143, 12396,  7285,  7048,  9409,  9919,
          8022,  9552,  9443, 22325, 39722, 16691, 10351,  9265,  8143, 14253,
         10173,  9599,  9265,  9134, 14253,  9445, 14160, 28292, 18160, 11638,
          9552, 27515,  7652, 23645,  9265,  7235, 1